In [1]:
import math
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
train = pd.read_csv('energy/train.csv', encoding='cp949')
test = pd.read_csv('energy/test.csv', encoding='cp949')

In [3]:
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']
    
for i in range(len(test)):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

In [4]:
def time(x):
    return int(x[-2:])

def weekday(x):
    return pd.to_datetime(x[:10]).weekday()

def month(x):
    return pd.to_datetime(x[:10]).month

train['hour']=train['date_time'].apply(lambda x: time(x))
test['hour']=test['date_time'].apply(lambda x: time(x))

train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))

train['month']=train['date_time'].apply(lambda x: month(x))
test['month']=test['date_time'].apply(lambda x: month(x))

test.interpolate(method='values', inplace=True)

train['date_time'] = pd.to_datetime(train['date_time'].apply(lambda x: str(x)+':00'))
test['date_time'] = pd.to_datetime(test['date_time'].apply(lambda x: str(x)+':00'))

In [7]:
#휴일여부

def holiday(x):
    if str(x.date()) == '2020-06-06' or str(x.date()) == '2020-08-15', or str(x.date()) == '2020-08-17':
        return 1
    elif x.weekday() == 5 or x.weekday() == 6:
        return 1
    else: 
        return 0

train['holiday'] = train['date_time'].apply(lambda x: holiday(x))
test['holiday'] = test['date_time'].apply(lambda x: holiday(x))

In [8]:
#체감온도, 불쾌지수

def get_pow(series):
    return math.pow(series, 0.15)

train['perceived_temperature'] = 13.12 + 0.6215*train['기온(°C)'] - 11.37*train['풍속(m/s)'].apply(get_pow) + 0.3965*train['풍속(m/s)'].apply(get_pow)*train['기온(°C)']
train['discomfort_index'] = 1.8*train['기온(°C)'] - 0.55*(1-train['습도(%)']/100)*(1.8*train['기온(°C)']-26) + 32

test['perceived_temperature'] = 13.12 + 0.6215*test['기온(°C)'] - 11.37*test['풍속(m/s)'].apply(get_pow) + 0.3965*test['풍속(m/s)'].apply(get_pow)*test['기온(°C)']
test['discomfort_index'] = 1.8*test['기온(°C)'] - 0.55*(1-test['습도(%)']/100)*(1.8*test['기온(°C)']-26) + 32

In [9]:
#강수량 encoding

train.iloc[:, 6] = train.iloc[:, 6].apply(lambda x: 0 if x == 0 else 1)
test.iloc[:, 5] = test.iloc[:, 5].apply(lambda x: 0 if x == 0 else 1)
test = test.rename(columns={'강수량(mm, 6시간)':'강수량(mm)', '일조(hr, 3시간)':'일조(hr)'})

In [10]:
# ref_day

def remove_date(x):
    if str(x)[:10] == '2020-06-01': return 0
    elif str(x)[:10] == '2020-06-02': return 0
    elif str(x)[:10] == '2020-06-03': return 0
    elif str(x)[:10] == '2020-06-04': return 0
    elif str(x)[:10] == '2020-06-05': return 0
    elif str(x)[:10] == '2020-06-06': return 0
    elif str(x)[:10] == '2020-06-07': return 0
#     elif str(x)[:10] == '2020-06-10': return 0
#     elif str(x)[:10] == '2020-06-11': return 0
#     elif str(x)[:10] == '2020-07-05': return 0
#     elif str(x)[:10] == '2020-07-12': return 0
#     elif str(x)[:10] == '2020-07-14': return 0
#     elif str(x)[:10] == '2020-07-27': return 0
#     elif str(x)[:10] == '2020-08-03': return 0
#     elif str(x)[:10] == '2020-08-08': return 0
#     elif str(x)[:10] == '2020-08-10': return 0
    else: return 1

# concat_df = pd.concat([train, test], axis=0).sort_values(by=['num', 'date_time']).reset_index(drop=True)
# concat_df['remove_day'] = concat_df['date_time'].apply(lambda x: remove_date(x))

train['remove_day'] = train['date_time'].apply(lambda x: remove_date(x))
concat_df = pd.concat([train, test], axis=0).sort_values(by=['num', 'date_time']).reset_index(drop=True)

df = concat_df[concat_df['remove_day'] != 0].reset_index(drop=True)
df = df.drop('remove_day', axis=1)

concat_df['date_time'] = concat_df['date_time'].apply(lambda x: x + timedelta(days=7))
ref_df = concat_df.iloc[:, :3].rename({'num':'num', 'date_time':'date_time', '전력사용량(kWh)':'ref_day'}, axis=1)

df = df.merge(ref_df, how='left', on=['num', 'date_time'])


In [11]:
#불쾌지수 구간화

bins = [0, 68, 75, 80, 100]

df['discomfort_index'] = pd.cut(df['discomfort_index'], bins, labels=[1, 2, 3, 4])

train = df[df.iloc[:, 2].notnull()].reset_index(drop=True)
test = df[df.iloc[:, 2].isnull()].reset_index(drop=True).drop('전력사용량(kWh)', axis=1)


cols = train.columns.tolist()[3:6] + ['perceived_temperature', 'ref_day', 'discomfort_index']

for i in cols:
    scaler = StandardScaler()
    scaler.fit(train[[i]])
    train[i] = scaler.transform(train[[i]])
    test[i] = scaler.transform(test[[i]])

In [12]:
# train = df[df.iloc[:, 2].notnull()].reset_index(drop=True)
# test = df[df.iloc[:, 2].isnull()].reset_index(drop=True).drop('전력사용량(kWh)', axis=1)

# cols = train.columns.tolist()[3:6] + ['perceived_temperature', 'discomfort_index']

# for i in cols:
#     scaler = MinMaxScaler()
#     scaler.fit(train[[i]])
#     train[i] = scaler.transform(train[[i]])
#     test[i] = scaler.transform(test[[i]])

In [13]:
train.to_csv('edit_train.csv', index=False)
test.to_csv('edit_test.csv', index=False)

- 6 10, 11  
- 7 5, 12, 14, 27  
- 8 3, 8, 10  

In [7]:
# cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# inertia_arr = []
# k_arr = range(2, 11)

# for k in k_arr:
#     kmeans = KMeans(n_clusters=k, init='k-means++', random_state=0)
#     kmeans.fit(cluster_df)
#     interia = kmeans.inertia_
    
#     print('k: {0}, inertia : {1:.4f}'.format(k, interia))
#     inertia_arr.append(interia)

# plt.plot(k_arr, inertia_arr)
# plt.xlabel('k')
# plt.ylabel('inertia')

**2040 = 85 * 24**

일자별 클러스터

In [14]:
# #num_hour_cluster군집화, 건물별 insert

# train_cluster_df = train['전력사용량(kWh)'].values.reshape(60, -1)

# kmeans = KMeans(n_clusters=10, init='k-means++', random_state=0)
# kmeans.fit(train_cluster_df)
# clusters = kmeans.predict(train_cluster_df)

# num_hour_df['num_hour_assignment'] = clusters
# train = train.merge(num_hour_df, how='left', on='num')
# test = test.merge(num_hour_df, how='left', on='num')